In [4]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from lesson_functions import *

## Template Matching

## Adjust Windows

## Multiple Detection (Heat-Map)

## Video Pipline